<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/21_Module_Life_Cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 21: Cykl życia nn.Module (__call__ vs forward)

Każda sieć w PyTorch dziedziczy po `nn.Module`. To nie jest zwykła klasa Pythona.
To **kontener**, który używa "czarnej magii" Pythona (`__setattr__`, `__call__`), żeby śledzić Twoje wagi.

**Kluczowa zasada:**
NIGDY nie wywołuj `model.forward(x)` ręcznie.
ZAWSZE wywołuj `model(x)`.

Dlaczego?
`__call__` (które jest wywoływane przez `model()`) robi mnóstwo rzeczy w tle:
1.  Uruchamia `_forward_pre_hooks`.
2.  Uruchamia `forward()`.
3.  Uruchamia `_forward_hooks`.

Jeśli wywołasz `forward` bezpośrednio, ominiesz system hooków (co zepsuje np. Profiling, Quantization i biblioteki typu Captum).

In [1]:
import torch
import torch.nn as nn

# 1. Definiujemy prosty moduł
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        print("   -> Wewnątrz forward()")
        return self.fc(x)

model = MyModule()
x = torch.randn(1, 10)

print("Model gotowy.")

Model gotowy.


## Eksperyment: Call vs Forward

Zarejestrujemy "Hooka" (funkcję, która odpala się automatycznie przy każdym przejściu danych).
Zobaczymy, że `forward()` go ignoruje.

In [2]:
# Funkcja-szpieg (Hook)
def spy_hook(module, input, output):
    print("🕵️ HOOK: Ktoś używa modelu!")

# Rejestrujemy hooka
handle = model.register_forward_hook(spy_hook)

print("--- 1. Użycie poprawne: model(x) ---")
# To wywołuje __call__
out1 = model(x)

print("\n--- 2. Użycie błędne: model.forward(x) ---")
# To omija __call__
out2 = model.forward(x)

print("\nWniosek: Widzisz? W drugim przypadku szpieg (Hook) nie zadziałał!")

--- 1. Użycie poprawne: model(x) ---
   -> Wewnątrz forward()
🕵️ HOOK: Ktoś używa modelu!

--- 2. Użycie błędne: model.forward(x) ---
   -> Wewnątrz forward()

Wniosek: Widzisz? W drugim przypadku szpieg (Hook) nie zadziałał!


## Magia Rejestracji (`__setattr__`)

W zwykłym Pythonie: `self.a = 5` po prostu przypisuje liczbę do obiektu.
W `nn.Module`: `self.layer = nn.Linear(...)` robi coś więcej.

PyTorch przechwytuje każde przypisanie (`__setattr__`).
1.  Sprawdza: "Czy to, co przypisujesz, to `Parameter` lub `Module`?".
2.  Jeśli tak: Dodaje to do specjalnej listy `_parameters` lub `_modules`.
3.  Dzięki temu `model.parameters()` lub `model.to('cuda')` wie, co ma przenieść, bez Twojej ingerencji.

In [3]:
class MagicModule(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 1. Zwykła zmienna Pythonowa (Ignorowana przez PyTorch)
        self.zwykla_zmienna = [1, 2, 3]
        
        # 2. Tensor (Też ignorowany! To częsty błąd!)
        self.zwykly_tensor = torch.randn(3, 3)
        
        # 3. nn.Parameter (To jest śledzone!)
        self.parametr = nn.Parameter(torch.randn(3, 3))
        
        # 4. Podmoduł (To też jest śledzone!)
        self.warstwa = nn.Linear(3, 3)

model_magic = MagicModule()

print("--- CO WIDZI PYTORCH? (model.state_dict()) ---")
# state_dict() zwraca tylko to, co PyTorch uznał za "swoje"
print(model_magic.state_dict().keys())

print("\n--- ANALIZA ---")
print("Widzisz 'parametr'? TAK.")
print("Widzisz 'warstwa.weight'? TAK.")
print("Widzisz 'zwykly_tensor'? NIE! (Nie zostanie zapisany przy save_model!)")

--- CO WIDZI PYTORCH? (model.state_dict()) ---
odict_keys(['parametr', 'warstwa.weight', 'warstwa.bias'])

--- ANALIZA ---
Widzisz 'parametr'? TAK.
Widzisz 'warstwa.weight'? TAK.
Widzisz 'zwykly_tensor'? NIE! (Nie zostanie zapisany przy save_model!)


## Pułapka Listy (`list` vs `nn.ModuleList`)

To jest błąd, który popełnia każdy junior.
Chcesz mieć listę 10 warstw. Piszesz:
`self.layers = [nn.Linear(...) for _ in range(10)]`

To **nie zadziała**. PyTorch nie zagląda do środka zwykłych list Pythona.
Te warstwy nie będą trenowane, nie trafią na GPU.

Musisz użyć **`nn.ModuleList`**.

In [4]:
class BrokenNet(nn.Module):
    def __init__(self):
        super().__init__()
        # ZŁE: Zwykła lista
        self.layers = [nn.Linear(10, 10) for _ in range(3)]

class FixedNet(nn.Module):
    def __init__(self):
        super().__init__()
        # DOBRE: ModuleList
        self.layers = nn.ModuleList([nn.Linear(10, 10) for _ in range(3)])

bad = BrokenNet()
good = FixedNet()

print(f"Liczba parametrów w BrokenNet: {len(list(bad.parameters()))}")
print("Wynik: 0. PyTorch 'nie widzi' warstw w liście.")

print(f"Liczba parametrów w FixedNet:  {len(list(good.parameters()))}")
print("Wynik: 6 (3 wagi + 3 biasy). Działa.")

Liczba parametrów w BrokenNet: 0
Wynik: 0. PyTorch 'nie widzi' warstw w liście.
Liczba parametrów w FixedNet:  6
Wynik: 6 (3 wagi + 3 biasy). Działa.


## 🥋 Black Belt Summary

1.  **Zasada nr 1:** Zawsze używaj `model(x)`, nigdy `model.forward(x)`.
2.  **Rejestracja:** Żeby tensor był "widziany" przez PyTorch (trening, save/load, GPU), musi być typu `nn.Parameter` lub być przypisany do `nn.Module`.
3.  **Kontenery:** Zwykła lista `[]` lub słownik `{}` ukrywają warstwy przed PyTorchem. Używaj `nn.ModuleList` i `nn.ModuleDict`.